In [26]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from typing import TypedDict, Annotated, List, Dict, Optional, Any
from pydantic import BaseModel, Field
import operator


In [27]:
load_dotenv()

True

In [28]:
model = ChatGoogleGenerativeAI(
    model = 'gemini-2.5-flash'
)

In [29]:
class EvaluationSchema(BaseModel):

    Feedback: str=Field(description="Detailed feedback for the content for linkedin")
    Score: str = Field(description = "Score out of 10", ge=0, le=10)


In [30]:
structured_llm = model.with_structured_output(EvaluationSchema)

In [ ]:
class PostDraft(TypedDict):

    """A single object that holds all the data for one LinkedIN post"""
    # Node 1 & 2
    original_topic : Dict[str, Any] # Dict[str, Any]: gives you the flexibility to handle this real-world, messy data while still having some type safety (you know the keys are strings).
    draft_text : str

    # Node 3
    fact_check: Optional[EvaluationSchema]
    quality_check: Optional[EvaluationSchema]
    engagement_check: Optional[EvaluationSchema]

    # Node 4 : final evaluation
    final_score : Optional[int]
    final_feedback : Optional[str]

    # Node 6 :Optimizer
    revision_count : int

class Published_post(TypedDict):

    post_id : str
    url : str
    published_at : str
    content_text : str

In [ ]:
class AutoInfluenceState(TypedDict):

    """The main state for autonomus LinkedIn agent"""

    # Node 8 (feedback) -> Node 1(Trend_analyzer)
    learned_preferances : Optional[str]

    # Node 1 -> Node 2(content generator)
    trending_topics : List[Dict[str, Any]]

    # This will contain LinkedIn drafts
    current_drafts : List[PostDraft]

    # Node 5(publisher) creates this 
    newly_published : List[Published_post]

    # Node 7(memory) adds this to master log 
    master_log_published : Annotated[List[Published_post], operator.add]

In [ ]:
# def node_name(state: AutoInfluenceState):
#     # read from state
#     # do something (API call, LLM, computation)
#     # return a partial dict of updates

In [ ]:
def trend_analyzer(state: AutoInfluenceState):

    return 

def content_generator(state: AutoInfluenceState):
    

    return 

In [ ]:
def fact_checker(state: AutoInfluenceState):
    
    return

def quality_checker(state: AutoInfluenceState):
    
    return

def engagement_checker(state: AutoInfluenceState):
    
    return

def final_evaluator(state: AutoInfluenceState):
    
    return


In [ ]:
def route_evaluation(state: AutoInfluenceState):

    drafts = state["current_drafts"]

    # If any draft passes the threshold -> Publish
    if any(d["final_score"] >= 8 for d in drafts) :
        return "publisher"
    
    # If any draft hits max revisions -> restart content generation
    elif any(d["revision_count"] >=3 for d in drafts):
        return "content_generator"
    
    # Otherwise -> keep optimizing
    else:
        return "optimizer"



In [ ]:
def optimizer(state: AutoInfluenceState):
    
    return

def publisher(state: AutoInfluenceState):
    
    return

def memory_logger(state: AutoInfluenceState):

    return

def feedback_updater(state: AutoInfluenceState):

    return

In [ ]:
# Now making the workflow
graph = StateGraph(AutoInfluenceState)

graph.add_node("trend_analyzer", trend_analyzer)
graph.add_node("content_generator", content_generator)

graph.add_edge(START, "trend_analyzer")
graph.add_edge("trend_analyzer", "content_generator")

graph.add_node("fact_checker", fact_checker)
graph.add_node("engagement_checker", engagement_checker)
graph.add_node("quality_checker", quality_checker)
graph.add_node("final_evaluator", final_evaluator) 

graph.add_edge("content_generator", "fact_checker")
graph.add_edge("content_generator", "enagaement_checker")
graph.add_edge("content_generator", "quality_checker")

graph.add_edge("fact_checker", "final_evaluator")
graph.add_edge("engagement_checker", "final_evaluator")
graph.add_edge("quality_checker", "final_evaluator")

graph.add_node("optimizer", optimizer) 
graph.add_node("publisher", publisher) 
graph.add_node("memory_logger", memory_logger) 
graph.add_node("feedback_updater", feedback_updater)

graph.add_conditional_edges("final_evaluator", route_evaluation)